# Zindi - Challenge

https://zindi.africa/competitions/predict-the-global-spread-of-covid-19

April 9, 2020

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib as plt 

## Import Data

In [34]:
#Read train 
deaths = pd.read_csv('data/deaths.csv')
cases = pd.read_csv('data/cases.csv')
deaths.head()

,Province/State,Country/Region,Territory,Population,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,...,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20
0,NaN,Afghanistan,Afghanistan,26023100.0,33.0000,65.0000,0,0,0,0,...,4,4,4,6,6,7,7,11,14,14
1,NaN,Albania,Albania,2895947.0,41.1533,20.1683,0,0,0,0,...,11,15,15,16,17,20,20,21,22,22
2,NaN,Algeria,Algeria,38700000.0,28.0339,1.6596,0,0,0,0,...,35,44,58,86,105,130,152,173,193,205
3,NaN,Andorra,Andorra,NaN,42.5063,1.5218,0,0,0,0,...,8,12,14,15,16,17,18,21,22,23
4,NaN,Angola,Angola,24383301.0,-11.2027,17.8739,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2


In [35]:
deaths.drop(['Province/State'], axis=True)

,Country/Region,Territory,Population,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,...,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20
0,Afghanistan,Afghanistan,26023100.0,33.000000,65.000000,0,0,0,0,0,...,4,4,4,6,6,7,7,11,14,14
1,Albania,Albania,2895947.0,41.153300,20.168300,0,0,0,0,0,...,11,15,15,16,17,20,20,21,22,22
2,Algeria,Algeria,38700000.0,28.033900,1.659600,0,0,0,0,0,...,35,44,58,86,105,130,152,173,193,205
3,Andorra,Andorra,NaN,42.506300,1.521800,0,0,0,0,0,...,8,12,14,15,16,17,18,21,22,23
4,Angola,Angola,24383301.0,-11.202700,17.873900,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2
5,Antigua and Barbuda,Antigua and Barbuda,86295.0,17.060800,-61.796400,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2
6,Argentina,Argentina,42669500.0,-38.416100,-63.616700,0,0,0,0,0,...,23,27,28,36,39,43,44,48,56,63
7,Armenia,Armenia,3009800.0,40.069100,45.038200,0,0,0,0,0,...,3,3,4,7,7,7,7,8,8,9
8,Australia,Australia,23696900.0,-35.473500,149.012400,0,0,0,0,0,...,1,1,1,1,1,2,2,2,2,2
9,Australia,Australia,23696900.0,-33.868800,151.209300,0,0,0,0,0,...,8,8,9,10,12,12,16,18,21,21


In [36]:
## Add many variables
cases.shape

(274, 83)

In [37]:
deaths.shape

(274, 84)

## Add variables


### Temperature 

In [7]:
import requests
import lxml.html as lh

In [38]:
def web_scraping_data(link):
    #Create a handle, page, to handle the contents of the website
    page = requests.get(link)
    doc = lh.fromstring(page.content)
    tr_elements = doc.xpath('//tr')

    tr_elements = doc.xpath('//tr')
    col=[]
    i=0
    for t in tr_elements[0]:
        i+=1
        name=t.text_content()
        #print ('%d:"%s"'%(i,name))
        col.append((name,[]))

    for j in range(1,len(tr_elements)):
        T=tr_elements[j]
        i=0

        for t in T.iterchildren():
            data=t.text_content() 
            if i>0:
                try:
                    data=int(data)
                except:
                    pass

            col[i][1].append(data)
            i+=1

    Dict={title:column for (title,column) in col}
    df=pd.DataFrame(Dict)
    
    return df

In [39]:
code_country = web_scraping_data("https://www.iban.com/country-codes")

In [40]:
code_country.head(2)

,Country,Alpha-2 code,Alpha-3 code,Numeric
0,Afghanistan,AF,AFG,4
1,Åland Islands,AX,ALA,248


In [41]:
code_country.shape

(249, 4)

Source 
- https://datacatalog.worldbank.org/dataset/climate-change-knowledge-portal-historical-data

In [85]:
temp = pd.read_csv('variable/temp.csv')

In [86]:
temp.shape

(178, 14)

In [87]:
temperature = pd.merge(left=temp, right=code_country, left_on='ISO_3DIGIT', right_on='Alpha-3 code')

In [88]:
temperature = temperature.drop(['Numeric','Alpha-2 code'], axis=True)

In [89]:
temperature.shape

(177, 16)

In [90]:
temperature.head(2)

,ISO_3DIGIT,Jan_Temp,Feb_temp,Mar_temp,Apr_Temp,May_temp,Jun_Temp,July_Temp,Aug_Temp,Sept_temp,Oct_temp,Nov_Temp,Dec_temp,Annual_temp,Country,Alpha-3 code
0,AFG,0.07,2.11,7.60,13.37,18.22,23.20,25.26,23.77,19.03,12.99,7.00,2.43,12.92,Afghanistan,AFG
1,AGO,22.58,22.68,22.78,22.35,20.74,18.37,17.95,19.90,22.19,23.18,22.79,22.61,21.51,Angola,AGO


In [91]:
len(deaths.Territory.unique())

209

In [92]:
deaths.shape

(274, 84)

In [93]:
death1 = pd.merge(left=deaths, right=temperature, left_on='Territory', right_on='Country')

In [94]:
death1.shape

(229, 100)

I am loosing 45 countries which are 

In [95]:
l1 = list(death1.Country)
l2 = list(deaths.Territory)
[x for x in l2 if x not in l1]

['Andorra',
 'Antigua and Barbuda',
 'Bahrain',
 'Barbados',
 'Cabo Verde',
 'Democratic Republic of the Congo (the)',
 'Faroe Islands',
 'Greenland',
 'French Polynesia',
 'Mayotte',
 'Reunion',
 'Saint Barthelemy',
 'St Martin',
 "Democratic People's Republic of Korea (the)",
 'Liechtenstein',
 'Maldives',
 'Malta',
 'Republic of Moldova (the)',
 'Monaco',
 'Aruba',
 'Curacao',
 'North Macedonia',
 'Saint Lucia',
 'San Marino',
 'Seychelles',
 'Singapore',
 'Taiwan',
 'United Republic of Tanzania (the)',
 'Bermuda',
 'Cayman Islands',
 'Montserrat',
 'Dominica',
 'Grenada',
 'Syrian Arab Republic (the)',
 'Saint Kitts and Nevis',
 'Sao Tome and Principe',
 'Kiribati',
 'Marshall Islands (the)',
 'Micronesia (Federated States of)',
 'Nauru',
 'Palau',
 'Republic of Korea (the)',
 'Samoa',
 'Tonga',
 'Tuvalu']

### Others epidemy 

SARS-CoV

China Canada, Hong Kong Special Administrative Region of China, Chinese Taipei, Singapore, and Hanoi in Viet Nam.

Ebola 

The Democratic Republic of the Congo
Gabon
Guinea
Italy
Ivory Coast
Liberia
Mali
Nigeria
Philippines
Russia
Senegal
Sierra Leone
South Africa
South Sudan
Spain
Uganda
United Kingdom
United States
What causes Ebol

### Age / density / wealth /sex 

https://datacatalog.worldbank.org/search?search_api_views_fulltext_op=AND&f%5B0%5D=field_collection_field%3A2026&sort_by=field_wbddh_modified_date

https://data.worldbank.org/indicator/en.pop.dnst

In [291]:
def wordbankdata(variable, tab):
    data = pd.read_csv('variable/'+variable+'.csv')
    data = data[['Country Code', '2018']]
    death = pd.merge(left=data, right=tab, left_on='Country Code', right_on='Alpha-3 code')
    death[variable] = death['2018']
    death.drop(['2018'], axis=True, inplace=True)
    return(death)

variable = '%+65'

In [292]:
death2 = wordbankdata(variable, death1)

In [293]:
l1 = list(death1.Country)
l2 = list(death2.Country)
[x for x in l1 if x not in l2]

['French Guiana', 'Guadeloupe']

In [295]:
variable = '15-64pop'
death2 = wordbankdata(variable, death2)
death2 = wordbankdata('density', death2)
death2 = wordbankdata( 'sex', death2)

In [298]:
rich = pd.read_csv('variable/income.csv')
rich = rich[['Country Code', 'IncomeGroup']]

In [299]:
death2 = pd.merge(left=rich, right=death2, left_on='Country Code', right_on='Alpha-3 code')

In [300]:
death2.drop(['Alpha-3 code','Country Code_y', 'Country Code_x','Country Code_y' ,'Country'], axis=1, inplace=True)


In [301]:
death2.head()

,IncomeGroup,Province/State,Country/Region,Territory,Population,Lat,Long,1/22/20,1/23/20,1/24/20,...,Aug_Temp,Sept_temp,Oct_temp,Nov_Temp,Dec_temp,Annual_temp,%+65,15-64pop,density,sex
0,Low income,NaN,Afghanistan,Afghanistan,26023100.0,33.0000,65.0000,0,0,0,...,23.77,19.03,12.99,7.00,2.43,12.92,2.584927,54.324898,56.937760,48.635847
1,Lower middle income,NaN,Angola,Angola,24383301.0,-11.2027,17.8739,0,0,0,...,19.90,22.19,23.18,22.79,22.61,21.51,2.216374,50.974702,24.713052,50.530463
2,Upper middle income,NaN,Albania,Albania,2895947.0,41.1533,20.1683,0,0,0,...,20.48,17.16,12.27,7.58,3.65,11.27,13.744736,68.582390,104.612263,49.063095
3,High income,NaN,United Arab Emirates,United Arab Emirates (the),NaN,24.0000,54.0000,0,0,0,...,33.55,31.74,28.34,24.06,20.28,26.83,1.085001,84.311490,135.609110,30.636688
4,Upper middle income,NaN,Argentina,Argentina,42669500.0,-38.4161,-63.6167,0,0,0,...,9.02,11.53,14.67,17.54,19.83,14.22,11.117789,64.121277,16.258510,51.237348


In [302]:
death2.shape

(227, 103)

In [303]:
deaths.shape

(274, 84)

## Split by continent

In [304]:
continent = pd.read_csv('variable/locations.csv')

In [305]:
continent.head()

,countriesAndTerritories,location,continent,population_year,population
0,Afghanistan,Afghanistan,Asia,2020.0,38928341.0
1,Albania,Albania,Europe,2020.0,2877800.0
2,Algeria,Algeria,Africa,2020.0,43851043.0
3,Andorra,Andorra,Europe,2020.0,77265.0
4,Angola,Angola,Africa,2020.0,32866268.0


In [306]:
continent.shape

(207, 5)

In [307]:
continent.drop(['population_year','population','countriesAndTerritories'], axis=1, inplace=True)

In [311]:
death2 = pd.merge(left=continent, right=death1, left_on='location', right_on='Country/Region')

In [312]:
death2.shape

(216, 102)

In [310]:
l1 = list(death4.location)
l2 = list(death3['Country/Region'])
[x for x in l2 if x not in l1]

[]

In [242]:
death4.head()

,location,continent,IncomeGroup,Province/State,Country/Region,Territory,Population,Lat,Long,1/22/20,...,July_Temp,Aug_Temp,Sept_temp,Oct_temp,Nov_Temp,Dec_temp,Annual_temp,%+65,Density,%-65
0,Afghanistan,Asia,Low income,NaN,Afghanistan,Afghanistan,26023100.0,33.0000,65.0000,0,...,25.26,23.77,19.03,12.99,7.00,2.43,12.92,2.584927,56.937760,54.324898
1,Albania,Europe,Upper middle income,NaN,Albania,Albania,2895947.0,41.1533,20.1683,0,...,20.54,20.48,17.16,12.27,7.58,3.65,11.27,13.744736,104.612263,68.582390
2,Algeria,Africa,Upper middle income,NaN,Algeria,Algeria,38700000.0,28.0339,1.6596,0,...,32.54,31.97,28.82,23.34,17.31,12.95,22.58,6.362497,17.730075,63.488825
3,Angola,Africa,Lower middle income,NaN,Angola,Angola,24383301.0,-11.2027,17.8739,0,...,17.95,19.90,22.19,23.18,22.79,22.61,21.51,2.216374,24.713052,50.974702
4,Argentina,South America,Upper middle income,NaN,Argentina,Argentina,42669500.0,-38.4161,-63.6167,0,...,7.42,9.02,11.53,14.67,17.54,19.83,14.22,11.117789,16.258510,64.121277


### Infected cases March 1 / March 15 / March 30

We take the number of cases every 15 days since the beginning of March. 

In [208]:
cases = pd.read_csv('data/cases.csv')

In [215]:
cases[['2/15/20', '3/1/20', '3/30/20', '3/30/20', 'Country/Region']]

,2/15/20,3/1/20,3/30/20,3/30/20,Country/Region
0,0,1,170,170,Afghanistan
1,0,0,223,223,Albania
2,0,1,584,584,Algeria
3,0,0,370,370,Andorra
4,0,0,7,7,Angola
5,0,0,7,7,Antigua and Barbuda
6,0,0,820,820,Argentina
7,0,1,482,482,Armenia
8,0,0,78,78,Australia
9,4,6,2032,2032,Australia


## Commorbidity / Risk Factors

### Obesity 

https://www.kaggle.com/arttua/who-obesity-by-country-2016

In [328]:
obesity = pd.read_csv('variable/obesity.csv')
obesity['Country'] = obesity['Unnamed: 0']
obesity.drop('Unnamed: 0', axis=1, inplace=True)

In [331]:
death3 = pd.merge(left=obesity, right=death2, left_on='Country', right_on='Country/Region')
death3.shape

In [334]:
death2.shape

(216, 102)

### 

## Medical care

### Hospital beds

https://www.kaggle.com/hamzael1/hospital-beds-by-country

In [341]:
beds = pd.read_csv('variable/beds.csv')

In [342]:
beds.count() / len(beds)

Country Name      1.000000
Country Code      1.000000
Indicator Name    1.000000
Indicator Code    1.000000
1960              0.613636
1961              0.007576
1962              0.007576
1963              0.007576
1964              0.007576
1965              0.030303
1966              0.018939
1967              0.018939
1968              0.018939
1969              0.018939
1970              0.662879
1971              0.030303
1972              0.034091
1973              0.034091
1974              0.041667
1975              0.340909
1976              0.056818
1977              0.041667
1978              0.045455
1979              0.045455
1980              0.492424
1981              0.280303
1982              0.113636
1983              0.098485
1984              0.125000
1985              0.378788
                    ...   
1991              0.363636
1992              0.318182
1993              0.412879
1994              0.318182
1995              0.318182
1996              0.443182
1

In [344]:
beds= beds[['Country Code', '2011']]

MERGE !

### Health Systems - WHO 


In [346]:
hs = pd.read_csv('variable/Health_systems.csv')

In [347]:
hs.shape

(210, 14)

## Strict mesure de confinement 

1 - 10

In [26]:
sum(deaths.Territory == 'France')

3

In [27]:
deaths[deaths.Territory == 'France']

,Province/State,Country/Region,Territory,Population,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,...,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20
114,Martinique,France,France,66078000.0,14.6415,-61.0242,0,0,0,0,...,1,3,3,3,3,3,4,4,4,6
115,NaN,France,France,66078000.0,46.2276,2.2137,0,0,0,0,...,3024,3523,4032,5387,6507,7560,8078,8911,10328,10869
253,Saint Pierre and Miquelon,France,France,66078000.0,46.8852,-56.3159,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
